# Implementar un MLP con PyTorch

<img src="figs/fig-MLP_XOR.png" width="50%">


1. **Definir la arquitectura de la red**:  
   - La red tendrá 2 entradas (los valores binarios del XOR), una capa oculta con 2 neuronas, y una neurona de salida.
   - Usar la función de activación sigmoide en las neuronas de la capa oculta y de salida.
   - Establecer una tasa de aprendizaje (ej. 0.5) y el número de épocas de entrenamiento.

   Por ejemplo, para la capa de salida (2 neuronas en la capa oculta, 1 neurona de salida):
 $$ W^{(2)} \in \mathbb{R}^{1 \times 2} $$
 $$ b^{(2)} \in \mathbb{R}^{1 \times 1} $$

2. **Inicializar los pesos y los sesgos**:  
   - Inicializar los pesos de las conexiones de la capa de entrada a la capa oculta y de la capa oculta a la capa de salida, de manera aleatoria (puedes usar la inicialización Xavier).
   - También inicializar los sesgos de cada capa.

3. **Propagación hacia adelante (Forward pass)**:  
   - Con la clase  $Linear$, se pueden definir las capas de la red. Aplica  una transformación lineal afín a los datos entrantes $ y =x W^T + b $  
   - Aplicar la función de activación (sigmoide) para obtener las activaciones de la capa oculta.
   - Repetir el proceso con los valores de las demás capas

4. **Calcular el error**:  
   - Calcular el error en la salida utilizando una función de error, como el Error Cuadrático Medio (MSE).

5. **Backpropagation (Propagación hacia atrás)**:  
   - Calcular los gradientes (backward)
   
6. **Actualizar de parámetros:  pesos y sesgos**:  
   - Usar los gradientes obtenidos para ajustar los pesos y los sesgos de la capa de salida y de la capa oculta utilizando el gradiente descendente.
   
7. **Repetir el entrenamiento**:  
   - Repetir los pasos de forward, cálculo de error, backpropagation y actualización de parámetros por el número de épocas definido hasta que el error disminuya significativamente.

8. **Evaluar el modelo**:  
   - Después del entrenamiento, probar la red con las entradas XOR y verificar que las salidas estén cerca de los valores esperados (0 o 1).
   


# Definición de los datos y minibatches

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Definimos los datos de entrada para XOR
X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]],  dtype=np.float32)


# Salidas esperadas para XOR
Y = np.array([[0], 
              [1], 
              [1], 
              [0]], dtype=np.float32)

# colocar la semilla para la generación de números aleatorios para la reproducibilidad de experimentos
random_state = 42
torch.manual_seed(random_state)
np.random.seed(random_state)


# Crear minibatches en PyTorch usando DataLoader
def create_minibatches(X, Y, batch_size):
    dataset = TensorDataset(X, Y) # Cargar los datos en un dataset de tensores
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader


# Uso de los minibatches

In [9]:
# Convertir datos a tensores de PyTorch
X_ = torch.tensor(X, dtype=torch.float32)
Y_ = torch.tensor(Y, dtype=torch.float32) 
dataL = create_minibatches(X_, Y_, batch_size=2)

In [10]:
# recorrer los minibatches
for a, b in dataL:
    print("====Mini batch=====")
    print(a, b, "\n")
    

====Mini batch=====
tensor([[0., 1.],
        [1., 1.]]) tensor([[1.],
        [0.]]) 

====Mini batch=====
tensor([[1., 0.],
        [0., 0.]]) tensor([[1.],
        [0.]]) 



# Definición de la arquitectura de la red

In [12]:

# Definir la red neuronal en PyTorch heredando de la clase base de Redes Neuronales: Module
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        # Definición de capas, funciones de activación e inicialización de pesos
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)

        if self.fc1.bias is not None:
            nn.init.zeros_(self.fc1.bias)
        if self.fc2.bias is not None:
            nn.init.zeros_(self.fc2.bias)        

    
    def forward(self, X):
        # Definición del orden de conexión de las capas y aplición de las funciones de activación
        out = self.fc1(X)
        out = self.sigmoid(out)  # Aplicamos la sigmoide en la capa oculta
        out = self.fc2(out)
        out = self.sigmoid(out)  # Aplicamos la sigmoide en la capa de salida
        return out

# Entrenamiento de la red

In [15]:


# Parámetros de la red
input_size = 2
hidden_size = 2 
output_size = 1
epochs = 10000
learning_rate = 0.2


# Convertir datos a tensores de PyTorch
X_train = torch.tensor(X, dtype=torch.float32)
Y_train = torch.tensor(Y, dtype=torch.float32)

# Crear la red
model = MLP(input_size, hidden_size, output_size)

# Definir la función de pérdida
# Mean Square Error (MSE)
criterion = nn.MSELoss()

# Definir el optimizador
#Parámetros del optimizador: parámetros del modelo y learning rate 
# Stochastic Gradient Descent (SGD)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Entrenamiento
print("Iniciando entrenamiento en PyTorch")

# Poner el modelo en modo de entrenamiento
model.train()  

for epoch in range(epochs):
    lossTotal = 0
    dataloader = create_minibatches(X_train, Y_train, batch_size=X_train.shape[0])
    for X_tr, y_tr in dataloader:
        # inicializar los gradientes en cero para cada época
        optimizer.zero_grad()
        
        # Propagación hacia adelante
        y_pred = model(X_tr)  #invoca al método forward de la clase MLP
        
        # Calcular el error MSE
        loss = criterion(y_pred, y_tr)
        #Acumular el error 
        lossTotal += loss.item()
        
        # Propagación hacia atrás: cálculo de los gradientes de los pesos y bias
        loss.backward()
        
        # actualización de los pesos: regla de actualización basado en el gradiente W = W - learning_rate * dE/dW
        optimizer.step()

    print(f"Época {epoch+1}/{epochs}, Pérdida: {lossTotal/len(dataloader)}")


Iniciando entrenamiento en PyTorch
Época 1/10000, Pérdida: 0.25030064582824707
Época 2/10000, Pérdida: 0.2502904534339905
Época 3/10000, Pérdida: 0.2502810060977936
Época 4/10000, Pérdida: 0.250272274017334
Época 5/10000, Pérdida: 0.2502642273902893
Época 6/10000, Pérdida: 0.25025674700737
Época 7/10000, Pérdida: 0.25024980306625366
Época 8/10000, Pérdida: 0.2502433955669403
Época 9/10000, Pérdida: 0.25023743510246277
Época 10/10000, Pérdida: 0.25023192167282104
Época 11/10000, Pérdida: 0.25022679567337036
Época 12/10000, Pérdida: 0.25022199749946594
Época 13/10000, Pérdida: 0.25021758675575256
Época 14/10000, Pérdida: 0.2502134144306183
Época 15/10000, Pérdida: 0.25020959973335266
Época 16/10000, Pérdida: 0.25020599365234375
Época 17/10000, Pérdida: 0.2502026855945587
Época 18/10000, Pérdida: 0.2501995265483856
Época 19/10000, Pérdida: 0.2501966059207916
Época 20/10000, Pérdida: 0.25019383430480957
Época 21/10000, Pérdida: 0.25019127130508423
Época 22/10000, Pérdida: 0.250188827514648

### Modo para predicción de datos

In [17]:
# Convertir los datos de prueba a tensores de PyTorch
X_test = torch.tensor(X_train, dtype=torch.float32)
y_test = Y_train

# Desactivar el comportamiento de modo de  entrenamiento: por ejemplo, capas como Dropout
model.eval()  # Establecer el modo del modelo a "evaluación"

with torch.no_grad():  # No  calcular gradientes 
    y_pred_test = model(X_test)

# y_test_pred contiene las predicciones
print("Predicciones:")
print(y_pred_test)

# Obtener la clase real

y_pred_test2 = torch.where(y_pred_test>=0.5, 1, 0)

print(y_pred_test2)

Predicciones:
tensor([[0.0648],
        [0.9387],
        [0.9388],
        [0.0671]])
tensor([[0],
        [1],
        [1],
        [0]])


/var/folders/by/nplj25n15fl9q9pvgqbdp2rw0000gn/T/ipykernel_2127/2740498151.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_train, dtype=torch.float32)
